# Implementing Boundary Conditions: From Linear to Nonlinear in FEM

In [13]:
using Ferrite
using SparseArrays
using Plots
using LinearAlgebra
using Gmsh
using FerriteGmsh 
using OrdinaryDiffEq 
using WriteVTK
using Tensors
using Printf

In [14]:
# deliberately use include to include code 
# see e.g. https://docs.julialang.org/en/v1/manual/code-loading/ 
include("flemish-fish.jl")

In [15]:
?fdmmesh

search: fdmmesh



Generates a one-dimensional uniform mesh between point 0 and 1

Input N(1) is number of elements. Output is the one-dimensional mesh. 

[`mesh(::NTuple{1,Int})`](@ref)

---

Generates a two-dimensional mesh between the points (0,0) and (1,1) 

Input N(1) and N(2) are the number of elements in x and y direction. Output is the two-dimensional mesh. 

[`mesh(::NTuple{2,Int})`](@ref)

---

Generates a two-dimensional mesh between the points a and b 

Input N(1) and N(2) are the number of elements in x and y direction. Output is the two-dimensional mesh. 

[`mesh(::NTuple{2,Int},::Point2D,::Point2D)`](@ref)


## Introduction 

This notebook outlines a structured approach to understanding and implementing boundary conditions (BCs) in numerical simulations. O

1.  **Hand-Coding Linear Dirichlet Boundary Conditions:** We will start by examining how existing libraries typically apply linear Dirichlet BCs, where a specific degree of freedom $\mathbf{u}_i$ is set to a fixed value $C$ (i.e., $\mathbf{u}_i = C$). Our focus will be on understanding how this condition translates into direct modifications of the global stiffness matrix $\mathbf{K}$ and the right-hand side vector $\mathbf{f}$. We will then proceed to "hand-code" this modification ourselves, learning how specific rows and columns of these matrices are altered to impose the constraint.

2.  **Implementing Affine Constraints:** Building upon the understanding of linear Dirichlet conditions, we will extend our capabilities to more general **affine constraints**. These are linear relationships involving multiple degrees of freedom, expressed as $\sum_{j} a_j \mathbf{u}_j = C$. A common example is enforcing a mean value constraint on a field variable over a certain region. This step will further develop our intuition for how linear dependencies are embedded within the algebraic system.

3.  **Extending to Nonlinear Boundary Conditions via Newton's Method:** Finally, we will apply the insights gained from handling linear constraints to the realm of nonlinear boundary conditions. For a nonlinear problem, the governing equations, including boundary conditions, form a system of nonlinear algebraic equations, $\mathbf{F}(\mathbf{u}) = \mathbf{0}$. We will solve this system using the Newton-Raphson method, which relies on solving a linearized system at each iteration: $J(\mathbf{u}^k) \mathbf{\delta u} = -\mathbf{F}(\mathbf{u}^k)$. Here, $J(\mathbf{u}^k)$ is the Jacobian matrix. Our key focus will be on how the nonlinear boundary condition (e.g., $[\rho_g(0)]^2 = 2$ from our previous work) is integrated into the residual function $\mathbf{F}(\mathbf{u})$ and, crucially, how its linearization appears within the Jacobian $J(\mathbf{u})$. This consolidates the understanding that even a nonlinear boundary condition translates to a linear contribution to the Jacobian system solved at each Newton iteration.

## Section 1 : Hand-Coded Linear Dirichlet Boundary Condition 

###  How linear Dirichlet boundary conditions are implemented using Ferrite ?

In this section, we will revisit our existing code from **section 4.1** of the "HydrogenProject" notebook : https://github.com/AnouchkaDESMETTRE/HydrogenProject/blob/main/HydrogenProject.ipynb, which addresses the steady-state gas density absorption equation:

$$D \frac{d^2 \rho_g}{dz^2} + u_z \frac{d \rho_g}{dz} + \dot{m} = 0$$

Our primary focus here will be to deeply understand how linear Dirichlet boundary conditions are implemented using Ferrite's `apply!(K,f,ch)` for FEM). Specifically, we will investigate how the system matrix $K$ and the right-hand side vector $f$ are directly modified. The goal is to explicitly "hand-code" the implementation of this Dirichlet boundary condition. This means we will bypass the high-level `applybc!` function for the Dirichlet constraint and instead directly manipulate the relevant row(s) of the discretized matrix and vector to enforce the condition $\rho_g(z_{boundary}) = C$. This exercise will provide a granular understanding of the algebraic changes required to impose such constraints.

In [49]:
# Problem parameters
L = 0.8
D = 1.0 
uz = 0.01
m_dot = 0.5
N = 5

interpolation = Lagrange{RefLine, 1}()
qr = QuadratureRule{RefLine}(2)
cell_values = CellValues(qr, interpolation)

# Définition précise du domaine
left = Vec((0.,))
right = Vec((L,))
grid = generate_grid(Ferrite.Line, (N,), left, right)

dh = DofHandler(grid)
add!(dh, :rho_g, interpolation)
close!(dh)

K = allocate_matrix(dh)
f = zeros(ndofs(dh))

function assemble!(K, f, cv, dh, D, uz, m_dot)
    assembler = start_assemble(K, f)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cv, cell)
        n_basefuncs = getnbasefunctions(cv)
        Ke = zeros(n_basefuncs, n_basefuncs)
        fe = zeros(n_basefuncs)
        for q in 1:getnquadpoints(cv)
            dΩ = getdetJdV(cv, q)
            for i in 1:n_basefuncs
                ϕ = shape_value(cv, q, i)
                ∇ϕ = shape_gradient(cv, q, i)[1]
                for j in 1:n_basefuncs
                    ∇ϕ_j = shape_gradient(cv, q, j)[1]
                    Ke[i, j] += (D * ∇ϕ * ∇ϕ_j - uz * ϕ * ∇ϕ_j) * dΩ
                end
                fe[i] += m_dot * ϕ * dΩ
            end
        end
        Ferrite.assemble!(assembler, celldofs(cell), Ke, fe)
    end
end

assemble!(K, f, cell_values, dh, D, uz, m_dot)

ch = ConstraintHandler(dh)
dbc1 = Dirichlet(:rho_g, [1], (x, t) -> 1.0) # Appliquer au nœud 1
dbc2 = Dirichlet(:rho_g, [Ferrite.getnnodes(grid)], (x, t) -> 0.0) # Appliquer au dernier nœud
add!(ch, dbc1)
add!(ch, dbc2)
close!(ch)

# --- Affichage AVANT apply! ---
println("Matrice K AVANT apply! (N=$N):")
display(Array(K)) # Convertir en Dense Array pour une meilleure visualisation des zéros
println("\n")
println("Vecteur f AVANT apply! (N=$N):")
display(f)
println("---------------------------------------------------------")

apply!(K, f, ch)

rho_g = K \ f

rho_g_computed_at_nodes = evaluate_at_grid_nodes(dh, rho_g, :rho_g);

# Génération des coordonnées z à partir des nœuds du maillage
z_coords = [grid.nodes[node].x[1] for node in 1:length(grid.nodes)];

## Initialiser le plot
#p_solution = plot(title="FEM Solution (N=$N)",
#    xlabel="z (m)", ylabel="ρg (kg/m³)", lw=2, legend=false) # legend=false car une seule courbe

## Plot de la solution FEM
#plot!(p_solution, z_coords, rho_g_computed_at_nodes, label="FEM Solution", lw=2, markershape=:circle, markersize=3)

## Afficher le plot
#display(p_solution)

Matrice K AVANT apply! (N=5):


6×6 Matrix{Float64}:
  6.255  -6.255   0.0     0.0     0.0     0.0
 -6.245  12.5    -6.255   0.0     0.0     0.0
  0.0    -6.245  12.5    -6.255   0.0     0.0
  0.0     0.0    -6.245  12.5    -6.255   0.0
  0.0     0.0     0.0    -6.245  12.5    -6.255
  0.0     0.0     0.0     0.0    -6.245   6.245



Vecteur f AVANT apply! (N=5):


6-element Vector{Float64}:
 0.039999999999999994
 0.07999999999999999
 0.07999999999999999
 0.07999999999999999
 0.08
 0.04

---------------------------------------------------------


**Stifness Matrix $K$ BEFORE `apply!`**

The values of the coefficients directly reflect the assembly of the element matrices for diffusion ($D/h$) and convection ($u\_z/2$).

* $K[1,1] = 6.255$ corresponds to the sum of diffusion and convection contributions for the first node: $\left(\frac{D}{h} + \frac{u_z}{2}\right) = (1.0/0.16 + 0.01/2) = 6.25 + 0.005 = 6.255$.

* Similarly, $K[1,2] = -6.255$ corresponds to $\left(-\frac{D}{h} - \frac{u_z}{2}\right) = (-1.0/0.16 - 0.01/2) = -6.25 - 0.005 = -6.255$.

* Diagonal terms for internal nodes, such as $K[2,2] = 12.5$, are approximately $\frac{2D}{h}$, as these nodes receive contributions from two adjacent elements ($2 \times 6.25 = 12.5$).

**Force Vector $f$ BEFORE `apply!`**

The values of the components are calculated by the integral $\int \dot{m} \phi_i \, d\Omega$.

* For the nodes at the domain boundaries (nodes 1 and 6), the contribution is $\dot{m} \cdot \frac{h}{2} = 0.5 \cdot (0.8/5) / 2 = 0.04$. This is observed for $f[6]$ and is very close for $f[1]$.

* For internal nodes (nodes 2, 3, 4, 5), the contribution is the sum of contributions from two adjacent elements, i.e., $\dot{m} \cdot h = 0.5 \cdot 0.16 = 0.08$. This is verified for $f[2]$ through $f[5]$.

In [50]:
# --- Affichage APRÈS apply! ---
println("Matrice K APRÈS apply! (N=$N):")
display(Array(K)) # Convertir en Dense Array
println("\n")
println("Vecteur f APRÈS apply! (N=$N):")
display(f)
println("\n")
# -----------------------------

Matrice K APRÈS apply! (N=5):


6×6 Matrix{Float64}:
 10.4167   0.0     0.0     0.0     0.0     0.0
  0.0     12.5    -6.255   0.0     0.0     0.0
  0.0     -6.245  12.5    -6.255   0.0     0.0
  0.0      0.0    -6.245  12.5    -6.255   0.0
  0.0      0.0     0.0    -6.245  12.5     0.0
  0.0      0.0     0.0     0.0     0.0    10.4167



Vecteur f APRÈS apply! (N=5):


6-element Vector{Float64}:
 10.416666666666664
  6.324999999999999
  0.07999999999999999
  0.07999999999999999
  0.08
  0.0

The applied Dirichlet boundary conditions are $\\rho\_g(z=0)=1.0$ (node 1) and $\\rho\_g(z=L)=0.0$ (node 6).

**Stifness Matrix $K$ AFTER `apply!`**

* The **first row** of $K$ (corresponding to node 1) has been significantly modified. $K[1,1]$ changed from $6.255$ to $10.4167$, and all other terms in this row ($K[1,2]$ to $K[1,6]$) became $0.0$.
  
* Similarly, the **last row** (corresponding to node 6) saw $K[6,6]$ change from $6.245$ to $10.4167$, with other terms in the row ($K[6,1]$ to $K[6,5]$) set to $0.0$.

* The **columns** corresponding to the constrained nodes (column 1 and column 6) have also been modified, with terms like $K[2,1]$ and $K[5,6]$ being set to $0.0$.

**Force Vector $f$ AFTER `apply!`**

* $f[1]$ changed from $0.0399...$ (source term) to $10.4166...$. This new value is the imposed boundary condition value at node 1 (which is $1.0$) multiplied by the factor applied to the diagonal of $K$ (here $10.4167$). Indeed, the equation for node 1 now effectively becomes $10.4167 \cdot \rho_{g,1} = 10.41666...$, which implies $\\rho\_{g,1} = 1.0$.

`apply!` implements strong Dirichlet boundary conditions using the *"large value on the diagonal method"*. This is a practical implementation of the "penalty method (see [1]) : 
For each constrained degree of freedom $i$ with imposed value $C$:
1.  Set $K[i,i]$ to a large factor (e.g., $10.4167$) and $K[i,j]$ to $0.0$ for $j \neq i$.
2.  Set $f[i]$ to (large factor) $\cdot C$.
3.  Adjust other $f[j]$ terms by transferring contributions from the constrained node ($f[j] = f[j] - K_{\text{original}}[j,i] \cdot C$).


* $f[6]$ changed from $0.04$ (source term) to $0.0$. Similarly, this is the BC value at node 6 (which is $0.0$) multiplied by the diagonal factor.

* Impact on Internal Nodes: The internal nodes ($f[3]$, $f[4]$, $f[5]$) retain their initial values from the source term, as their equations are not directly affected by this "row/column modification" method for the boundary nodes.

* However, $f[2]$ is affected by the treatment of the boundary condition at node 1. This is due to the "transfer" of contributions from the constrained node to the right-hand side of the equations for adjacent unconstrained nodes. 

Before applying the Dirichlet boundary condition at node 1, the second equation in our linear system $\mathbf{K}\mathbf{u} = \mathbf{f}$ (corresponding to node 2) would look like this:

$$K[2,1]\rho_{g,1} + K[2,2]\rho_{g,2} + K[2,3]\rho_{g,3} = f_{\text{original}}[2]$$

where $f_{\text{original}}[2]$ is the value on the right-hand side for node 2, arising solely from the integration of the source term $\dot{m}$.

When we impose a Dirichlet boundary condition $\rho_{g,1} = C_1$ (in our case, $C_1 = 1.0$), the value of $\rho_{g,1}$ is now known. The strategy used by the `apply!` function (and common in Finite Element Method implementations) is to "remove" the terms whose values are already known from the left-hand side of the equations (see [2]). The term $K[2,1]\rho_{g,1}$ is a contribution whose value is known (since $K[2,1]$ is a matrix coefficient and $\rho_{g,1}$ is our imposed value $C_1$). This known contribution is then moved from the left-hand side to the right-hand side of the equation : 

$$K[2,2]\rho_{g,2} + K[2,3]\rho_{g,3} = f_{\text{original}}[2] - K_{\text{original}}[2,1]\rho_{g,1}$$

Thus, the new value of $f[2]$ (let's call it $f_{\text{new}}[2]$) is calculated as follows:

$$f_{\text{new}}[2] = f_{\text{original}}[2] - K_{\text{original}}[2,1] \cdot C_1$$

Let's verify this with the values we obtained in our output:
* $f_{\text{original}}[2] = 0.07999999999999999 \approx 0.08$
* $K_{\text{original}}[2,1] = -6.245$
* $C_1 = 1.0$ (the value of $\rho_g$ at node 1 imposed by the BC)

Substituting these values:
$$f_{\text{new}}[2] = 0.08 - (-6.245) \cdot 1.0$$
$$f_{\text{new}}[2] = 0.08 + 6.245 = 6.325$$

And $f[2]$ after `apply!` is $6.324999999999999$, which is almost exactly $6.325$.



[1] Dr.Y.DILIPKUMAR (2025). Finite Element Analysis [https://mrcet.com/downloads/digital_notes/ME/IV%20year/Finite%20Element%20Analysis.pdf](https://mrcet.com/downloads/digital_notes/ME/IV%20year/Finite%20Element%20Analysis.pdf)]

[2] : Pr. S. Deparis. (2004). Numerical Analysis of Axisymmetric Flows and Methods for Fluid-Structure Interaction Arising in Blood Flow Simulation (Thèse No 2965). École Polytechnique Fédérale de Lausanne (EPFL)

### Manual Implementation of Linear Dirichlet Boundary Conditions

The observation of `apply!`'s effect on $K$ and $f$ reveals the commonly used strategy for implementing linear Dirichlet boundary conditions (where a value is imposed for a Degree of Freedom). To implement this "by hand" in our code, we must follow these steps for each DoF $i$ where a value $C$ is imposed ($u_i = C$):

1.  **Modify Row $i$ of the Stiffness Matrix $K$:**

      * Set the diagonal value $K[i,i]$ to a non-zero value. This is a "large value" (as observed with Ferrite, e.g., $10.4167$) to ensure numerical stability and diagonal dominance, especially when dealing with floating-point arithmetic.
      
      * Set all other elements in row $i$ (i.e., $K[i,j]$ for $j \neq i$) to $0.0$.

2.  **Modify Column $i$ of the Stiffness Matrix $K$:**

      * For each row $j$ where $j \neq i$, set the off-diagonal element $K[j,i]$ to $0.0$. This ensures that the equation for node $j$ does not explicitly depend on the unknown value of $u_i$ (which is already known and imposed).

3.  **Modify the Force Vector $f$:**

      * Set the element $f[i]$ to the imposed value $C$. If a "large value" factor (like $10.4167$) was used for $K[i,i]$, then $f[i]$ should be set to $\text{factor} \cdot C$. This ensures that the $i$-th equation becomes $\text{factor} \cdot u_i = \text{factor} \cdot C$, directly forcing $u_i= C$.
      
      * If we performed step 2 (setting column $i$ to zero), we must transfer the contribution of the known boundary condition value $C$ to the right-hand side (vector $f$) of the equations for other nodes. For each row $j$ where $j \neq i$:
        $$f[j]_{\text{new}} = f[j]_{\text{original}} - K_{\text{original}}[j,i] \cdot C$$
        where $K\_{\text{original}}[j,i]$ is the value of the matrix element before setting $K[j,i]$ to zero. This precisely accounts for the known value of $u_i$ in the equations of connected nodes.

These steps ensure that the equation for the constrained degree of freedom becomes trivially satisfied by the imposed value, while its influence on other equations is correctly accounted for, leading to a well-posed system for the remaining unknown degrees of freedom.

In [51]:
# Problem parameters
L = 0.8
D = 1.0
uz = 0.01
m_dot = 0.5
N = 5 # Nombre d'éléments pour une visualisation facile

interpolation = Lagrange{RefLine, 1}()
qr = QuadratureRule{RefLine}(2)
cell_values = CellValues(qr, interpolation)

# Définition précise du domaine
left = Vec((0.,))
right = Vec((L,))
grid = generate_grid(Ferrite.Line, (N,), left, right)

dh = DofHandler(grid)
add!(dh, :rho_g, interpolation)
close!(dh)

K = allocate_matrix(dh) # K sera une SparseMatrixCSC
f = zeros(ndofs(dh))

function assemble!(K, f, cv, dh, D, uz, m_dot)
    assembler = start_assemble(K, f)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cv, cell)
        n_basefuncs = getnbasefunctions(cv)
        Ke = zeros(n_basefuncs, n_basefuncs)
        fe = zeros(n_basefuncs)
        for q in 1:getnquadpoints(cv)
            dΩ = getdetJdV(cv, q)
            for i in 1:n_basefuncs
                ϕ = shape_value(cv, q, i)
                ∇ϕ = shape_gradient(cv, q, i)[1]
                for j in 1:n_basefuncs
                    ∇ϕ_j = shape_gradient(cv, q, j)[1]
                    Ke[i, j] += (D * ∇ϕ * ∇ϕ_j - uz * ϕ * ∇ϕ_j) * dΩ
                end
                fe[i] += m_dot * ϕ * dΩ
            end
        end
        Ferrite.assemble!(assembler, celldofs(cell), Ke, fe)
    end
end

assemble!(K, f, cell_values, dh, D, uz, m_dot)

# --- DÉBUT DE L'APPLICATION MANUELLE DES CONDITIONS LIMITES ---

# Stocker une copie de K avant modification pour le transfert des contributions à f
K_original = deepcopy(K)

# Définir les conditions limites (nœud -> valeur)
# Nœud 1 (z=0) : rho_g = 1.0
# Nœud N+1 (z=L) : rho_g = 0.0
constrained_dofs = [1, Ferrite.getnnodes(grid)] # Indices des nœuds contraints
constrained_values = [1.0, 0.0] # Valeurs imposées

# Facteur "large valeur" observé avec Ferrite pour N=5
# (Nous utilisons la valeur observée pour correspondre au comportement de Ferrite)
large_factor = 10.4167

# Appliquer les conditions limites manuellement
for (i, dof_idx) in enumerate(constrained_dofs)
    bc_value = constrained_values[i]

    # 1. Transférer les contributions des nœuds contraints vers le vecteur f
    # Pour chaque autre nœud j, soustraire K_original[j, dof_idx] * bc_value de f[j]
    for j in 1:ndofs(dh)
        if j != dof_idx
            f[j] -= K_original[j, dof_idx] * bc_value
        end
    end

    # 2. Modifier la matrice K pour le nœud contraint 
    # D'abord, mettre à zéro toutes les entrées *hors diagonale* de la ligne dof_idx
    for col_idx in 1:ndofs(dh)
        if col_idx != dof_idx # Ne pas toucher l'élément diagonal pour l'instant
            K[dof_idx, col_idx] = 0.0
        end
    end
    # Ensuite, mettre à zéro toutes les entrées *hors diagonale* de la colonne dof_idx
    for row_idx in 1:ndofs(dh)
        if row_idx != dof_idx # Ne pas toucher l'élément diagonal pour l'instant
            K[row_idx, dof_idx] = 0.0
        end
    end
    # Enfin, définir la "grande valeur" sur l'élément diagonal
    # Cet élément est ré-ajouté ou mis à jour dans la structure sparse.
    K[dof_idx, dof_idx] = large_factor

    # 3. Modifier l'élément f[dof_idx] du vecteur f
    f[dof_idx] = large_factor * bc_value
end

# --- Affichage APRÈS application manuelle des CL ---
println("Matrice K APRÈS application manuelle des CL (N=$N):")
display(Array(K)) # Convertir en Dense Array
println("\n")
println("Vecteur f APRÈS application manuelle des CL (N=$N):")
display(f)

Matrice K APRÈS application manuelle des CL (N=5):


6×6 Matrix{Float64}:
 10.4167   0.0     0.0     0.0     0.0     0.0
  0.0     12.5    -6.255   0.0     0.0     0.0
  0.0     -6.245  12.5    -6.255   0.0     0.0
  0.0      0.0    -6.245  12.5    -6.255   0.0
  0.0      0.0     0.0    -6.245  12.5     0.0
  0.0      0.0     0.0     0.0     0.0    10.4167



Vecteur f APRÈS application manuelle des CL (N=5):


6-element Vector{Float64}:
 10.4167
  6.324999999999999
  0.07999999999999999
  0.07999999999999999
  0.08
  0.0

## Section 2: Affine Constraints as Boundary Conditions 

Building upon our understanding of imposing linear Dirichlet conditions, we now extend our capabilities to more complex scenarios involving affine constraints. These are linear relationships between multiple degrees of freedom, expressed generally as $\sum_{j} a_j u_j = C$. To illustrate this, we will solve the incompressible Stokes equations in a 2D rectangular domain.

The Stokes equations describe the steady, laminar flow of viscous, incompressible fluids at low Reynolds numbers. They consist of two main parts:

1.  **Momentum Equation:** This describes the balance of forces within the fluid.
    $$-\nabla p + \mu \nabla^2 \mathbf{u} = \mathbf{0}$$
    where $p$ is the pressure scalar, $\mathbf{u}$ is the velocity vector (with components $(u_x, u_y)$ in 2D), and $\mu$ is the dynamic viscosity.

2.  **Continuity Equation:** This enforces mass conservation for incompressible flows.
    $$\nabla \cdot \mathbf{u} = 0$$

For our problem, we will consider a rectangular domain $\Omega$. The boundary conditions will be defined as follows:

* **Inlet Velocity:** On the left boundary ($\Gamma_{\text{left}}$), we impose a constant velocity profile, for example, $\mathbf{u} = (U_0, 0)$ where $U_0$ is a constant. This is a strong Dirichlet condition on velocity.

* **No-Slip Walls:** On the remaining boundaries ($\Gamma_{\text{top}}$, $\Gamma_{\text{bottom}}$, and $\Gamma_{\text{right}}$), we apply a no-slip condition, meaning the fluid velocity is zero: $\mathbf{u} = \mathbf{0}$. This is also a strong Dirichlet condition on velocity.

* **Mean Zero Pressure:** To uniquely determine the pressure field (which is only defined up to an arbitrary constant in incompressible flows), we impose a **mean average zero condition on pressure over the entire boundary** ($\partial \Omega$). This is expressed as:
    $$\int_{\partial \Omega} p \, ds = 0$$
    This particular condition serves as our primary example for implementing an **affine constraint**, as it involves an integral relationship over multiple pressure degrees of freedom on the boundary.

We are using the tutorial Ferrite on Stokes as a reference : https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/stokes-flow/

In [4]:
grid = togrid("long_rectangular_mesh.msh")

Info    : Reading 'long_rectangular_mesh.msh'...
Info    : 9 entities
Info    : 1111 nodes
Info    : 1220 elements
Info    : Done reading 'long_rectangular_mesh.msh'


Grid{2, Quadrilateral, Float64} with 1000 Quadrilateral cells and 1111 nodes

<img src="./tank_mesh.png" width=800 />

In [5]:
# Adapted setup_fevalues for Quadrilateral elements
function setup_fevalues(ipu, ipp, ipg)
    qr = QuadratureRule{RefQuadrilateral}(2)
    cvu = CellValues(qr, ipu, ipg)
    cvp = CellValues(qr, ipp, ipg)
    qr_facet = FacetQuadratureRule{RefQuadrilateral}(2)
    fvp = FacetValues(qr_facet, ipp, ipg)
    return cvu, cvp, fvp
end

# setup_dofs function remains generic, no changes needed.
function setup_dofs(grid, ipu, ipp)
    dh = DofHandler(grid)
    add!(dh, :u, ipu) # Velocity field
    add!(dh, :p, ipp) # Pressure field
    close!(dh)
    return dh
end

# Adapted setup_mean_constraint for new faceset names and quadrilateral elements
function setup_mean_constraint(dh, fvp)
    assembler = Ferrite.COOAssembler()
    # All external boundaries for the mean pressure constraint
    set = union(
        getfacetset(dh.grid, "Gamma_left"),
        getfacetset(dh.grid, "Gamma_right"),
        getfacetset(dh.grid, "Gamma_top"),
        getfacetset(dh.grid, "Gamma_bottom"),
    )
    # Allocate buffers
    range_p = dof_range(dh, :p)
    element_dofs = zeros(Int, ndofs_per_cell(dh))
    element_dofs_p = view(element_dofs, range_p)
    element_coords = zeros(Vec{2}, 4) # Changed from 3 to 4 for quadrilaterals
    Ce = zeros(1, length(range_p)) # Local constraint matrix (only 1 row)
    # Loop over all the boundaries
    for (ci, fi) in set
        Ce .= 0
        getcoordinates!(element_coords, dh.grid, ci)
        reinit!(fvp, element_coords, fi)
        celldofs!(element_dofs, dh, ci)
        for qp in 1:getnquadpoints(fvp)
            dΓ = getdetJdV(fvp, qp)
            for i in 1:getnbasefunctions(fvp)
                Ce[1, i] += shape_value(fvp, qp, i) * dΓ
            end
        end
        # Assemble to row 1
        assemble!(assembler, [1], element_dofs_p, Ce)
    end
    C, _ = finish_assemble(assembler)

    # Create an AffineConstraint from the C-matrix
    _, J, V = findnz(C)

    if isempty(V)
        @warn "No pressure DOFs found on boundary for mean constraint. Constraint might be trivial or misconfigured."
        return nothing
    end
    _, constrained_dof_idx = findmax(abs, V) # Use abs for robustness with negative coefficients
    constrained_dof = J[constrained_dof_idx]

    constrained_coefficient = V[constrained_dof_idx]
    if abs(constrained_coefficient) < eps(Float64)
        error("Coefficient for chosen constrained DOF ($constrained_dof) is too small, cannot normalize.")
    end

    pairs = Pair{Int, Float64}[]
    for k in 1:length(J)
        if J[k] != constrained_dof
            push!(pairs, J[k] => -V[k] / constrained_coefficient)
        end
    end

    mean_value_constraint = AffineConstraint(
        constrained_dof,
        pairs,
        0.0 / constrained_coefficient, # Still 0.0 (mean value should be zero)
    )
    return mean_value_constraint
end

# Adapted setup_constraints for rectangular geometry BCs and laminar flow inlet
function setup_constraints(dh, fvp, U_max_inlet::Float64, Ly_channel::Float64) # Added U_max_inlet and Ly_channel as parameters
    ch = ConstraintHandler(dh)

    # 1. Inlet velocity (Dirichlet BC on Gamma_left) - Parabolic Laminar Flow Profile
    inlet_tag = getfacetset(dh.grid, "Gamma_left")

    # English comment explaining laminar flow profile:
    # The inlet velocity on 'Gamma_left' (x=0) is prescribed using a parabolic (Poiseuille) profile,
    # characteristic of laminar flow in a channel. The maximum velocity (U_max_inlet) is at the
    # channel centerline (y = Ly_channel/2), and it is zero at the top and bottom walls (y=0, y=Ly_channel).
    laminar_flow_profile = (x_spatial, t) -> begin
        y = x_spatial[2] # Extract y-coordinate from the spatial point
        ux = U_max_inlet * 4.0 * y * (Ly_channel - y) / (Ly_channel^2)
        return Vec{2}((ux, 0.0)) # Velocity vector (ux, uy=0)
    end
    dbc_inlet = Dirichlet(:u, inlet_tag, laminar_flow_profile, [1, 2]) # Apply to both x and y components
    add!(ch, dbc_inlet)

    # 2. No-slip conditions on the rest of the walls
    # Gamma_top, Gamma_bottom, and Gamma_right boundaries
    no_slip_tags = union(
        getfacetset(dh.grid, "Gamma_top"),
        getfacetset(dh.grid, "Gamma_bottom"),
        getfacetset(dh.grid, "Gamma_right"), # Right boundary as no-slip (closed box)
    )
    dbc_noslip = Dirichlet(:u, no_slip_tags, (x, t) -> Vec{2}((0.0, 0.0)), [1, 2]) # [0,0] velocity
    add!(ch, dbc_noslip)

    # 3. Mean value constraint for pressure on ALL boundaries
    mean_value_constraint = setup_mean_constraint(dh, fvp)
    if mean_value_constraint !== nothing
        add!(ch, mean_value_constraint)
    else
        @warn "Mean pressure constraint was not added, check setup_mean_constraint."
    end

    # Finalize
    close!(ch)
    update!(ch, 0) # Update for time t=0 (if needed for time-dependent BCs)
    return ch
end

# Assemble function for Stokes equations (modified to remove body force and add viscosity)
function assemble_system!(K, f, dh, cvu, cvp, viscosity::Float64) # Added viscosity parameter
    assembler = start_assemble(K, f)

    ke = zeros(ndofs_per_cell(dh), ndofs_per_cell(dh))
    fe = zeros(ndofs_per_cell(dh))

    # Get dof ranges for velocity (u) and pressure (p)
    range_u = dof_range(dh, :u)
    ndofs_u_cell = length(range_u) # Number of velocity dofs per cell
    range_p = dof_range(dh, :p)
    ndofs_p_cell = length(range_p) # Number of pressure dofs per cell

    # Buffers for shape functions and gradients
    ϕᵤ = Vector{Vec{2, Float64}}(undef, ndofs_u_cell)
    ∇ϕᵤ = Vector{Tensor{2, 2, Float64, 4}}(undef, ndofs_u_cell)
    divϕᵤ = Vector{Float64}(undef, ndofs_u_cell)
    ϕₚ = Vector{Float64}(undef, ndofs_p_cell)

    for cell in CellIterator(dh)
        reinit!(cvu, cell)
        reinit!(cvp, cell)

        ke .= 0.0 # Reset elemental stiffness matrix
        fe .= 0.0 # Reset elemental force vector

        for qp in 1:getnquadpoints(cvu) # Assuming cvu and cvp have same quadrature points
            dΩ = getdetJdV(cvu, qp) # Differential volume element

            # Pre-calculate shape functions and gradients at current quadrature point
            for i in 1:ndofs_u_cell
                ϕᵤ[i] = shape_value(cvu, qp, i)
                ∇ϕᵤ[i] = shape_gradient(cvu, qp, i)
                divϕᵤ[i] = shape_divergence(cvu, qp, i)
            end

            for i in 1:ndofs_p_cell
                ϕₚ[i] = shape_value(cvp, qp, i)
            end

            # Assemble elemental stiffness matrix (ke)
            # u-u block (viscous term: μ * ∇u : ∇v)
            for (i, I) in pairs(range_u), (j, J) in pairs(range_u)
                ke[I, J] += (viscosity * (∇ϕᵤ[i] ⊡ ∇ϕᵤ[j])) * dΩ
            end

            # u-p block (pressure gradient term: -p * div(u))
            for (i, I) in pairs(range_u), (j, J) in pairs(range_p)
                ke[I, J] += (-divϕᵤ[i] * ϕₚ[j]) * dΩ
            end

            # p-u block (continuity term: -div(u) * p)
            for (i, I) in pairs(range_p), (j, J) in pairs(range_u)
                ke[I, J] += (-ϕₚ[i] * divϕᵤ[j]) * dΩ
            end

            # No p-p block for standard Stokes (incompressible)
            # No body force term
        end
        # Assemble local to global
        assemble!(assembler, celldofs(cell), ke, fe)
    end
    return K, f
end


function main(grid::Grid) # La fonction main prend maintenant la grille comme argument
    # Fluid properties
    rho = 1.0       # Fluid density
    viscosity = 1.0 # Dynamic viscosity

    # Ces paramètres DOIVENT correspondre aux dimensions de la grille chargée
    # Ils sont utilisés pour le calcul du Reynolds et le profil d'entrée
    Lx = 1.0 # Longueur du canal
    Ly = 0.1 # Hauteur du canal
    h = 0.01 # Longueur caractéristique (pour le nom du fichier de sortie)

    # Desired flow regime: Target Reynolds Number
    # For 2D channel flow (Poiseuille), laminar flow is typically Re < ~2300
    target_reynolds_number = 100.0 # Example: A clearly laminar flow

    # Calculate U_max_inlet based on target Reynolds number
    # Re = (rho * U_avg * Ly) / viscosity  => U_avg = (Re * viscosity) / (rho * Ly)
    U_avg_calculated = (target_reynolds_number * viscosity) / (rho * Ly)

    # For Poiseuille flow in a 2D channel, U_max = 1.5 * U_avg
    U_max_inlet_calculated = 1.5 * U_avg_calculated

    println("Calculated U_max_inlet for Re = $(target_reynolds_number): $(U_max_inlet_calculated)")

    # Interpolations (Taylor-Hood Q2-Q1 for Quadrilateral elements)
    # Velocity: Quadratic on Quads, 2 components
    ipu = Lagrange{RefQuadrilateral, 2}()^2
    # Pressure: Linear on Quads, 1 component
    ipp = Lagrange{RefQuadrilateral, 1}()

    # Dofs
    dh = setup_dofs(grid, ipu, ipp)

    # FE values
    ipg = Lagrange{RefQuadrilateral, 1}() # linear geometric interpolation
    cvu, cvp, fvp = setup_fevalues(ipu, ipp, ipg)

    # Boundary conditions
    ch = setup_constraints(dh, fvp, U_max_inlet_calculated, Ly) # Pass calculated U_max_inlet and Ly

    # Global tangent matrix and rhs
    coupling = [true true; true false] # No coupling between pressure test/trial functions
    K = allocate_matrix(dh, ch; coupling = coupling)
    f = zeros(ndofs(dh))

    # Assemble system
    assemble_system!(K, f, dh, cvu, cvp, viscosity) # Pass viscosity

    # Apply boundary conditions and solve
    apply!(K, f, ch) # Apply Dirichlet and affine constraints to K and f
    u = K \ f         # Solve the linear system
    apply!(u, ch)     # Apply constraints to the solution vector itself (e.g., for affine constraint substitution)

    # Export the solution (velocity and pressure)
    # Le nom de fichier est adapté pour le maillage long et fin
    VTKGridFile("stokes_long_rectangular_laminar_flow_Re$(Int(target_reynolds_number))_h$(replace(string(h), "." => "_"))", grid) do vtk
        # Velocity is a vector field, Pressure is a scalar field
        write_solution(vtk, dh, u, :u)
        write_solution(vtk, dh, u, :p)
    end

    println("Stokes flow simulation (long rectangular laminar flow) complete. VTK output: stokes_long_rectangular_laminar_flow_Re$(Int(target_reynolds_number))_h$(replace(string(h), "." => "_")).vtu")
    println("Open with ParaView to visualize velocity vectors and pressure field.")

    return
end

# Ensuite, appelez la fonction main avec la grille chargée:
main(grid)

Calculated U_max_inlet for Re = 100.0: 1500.0


LoadError: UndefVarError: `reinit!` not defined in `Main`
Hint: It looks like two or more modules export different bindings with this name, resulting in ambiguity. Try explicitly importing it from a particular module, or qualifying the name with the module it should come from.
Hint: a global variable of this name also exists in Ferrite.
Hint: a global variable of this name also exists in SciMLBase.
Hint: a global variable of this name also exists in DiffEqBase.
Hint: a global variable of this name may be made accessible by importing LinearSolve in the current active module Main
Hint: a global variable of this name may be made accessible by importing OrdinaryDiffEqExtrapolation in the current active module Main
Hint: a global variable of this name may be made accessible by importing OrdinaryDiffEqStabilizedRK in the current active module Main
Hint: a global variable of this name may be made accessible by importing OrdinaryDiffEqStabilizedIRK in the current active module Main
Hint: a glo

## Section 3: Non-Linear Case 

This section addresses the challenge of accurately solving the 1D heat equation with a nonlinear boundary condition from $t=0$ onwards. Our previous attempts with a pseudo-transient approach involving a penalty method doesn't make the solution at $t=0$ accurate : https://github.com/AnouchkaDESMETTRE/HydrogenProject/blob/main/pressure-inlet-conditions.ipynb 

Our objective is to solve this time-dependent problem by directly incorporating the nonlinear boundary condition into the global finite element system, allowing for a precise solution at each time step, including $t=0$. This approach aims for results comparable to commercial software like OpenFOAM.

### 3.1 1D Heat Equation : Problem Formulation

We consider the following 1D heat diffusion equation:

$$\frac{\partial u}{\partial t} - \frac{d}{dx} \left( k \frac{du}{dx} \right) = 0$$

where:
* $u(x, t)$ is the temperature field.
* $k$ is the thermal conductivity (constant).

**Boundary Conditions**

1.  Left Boundary ($x=0$) - Nonlinear Dirichlet (Directly Enforced):
    $$(u(0, t))^2 = 100$$
    This condition introduces a nonlinearity. Instead of a penalty method, we will directly modify the residual and Jacobian equations corresponding to the degrees of freedom (DOFs) at $x=L$ to enforce this relationship. This transforms the problem at each time step into a nonlinear algebraic system.
    
3.  Right Boundary ($x=L$) - Homogeneous Dirichlet:
    $$u(L, t) = 0$$
    This is a standard linear Dirichlet boundary condition, enforced strongly.


**Initial Condition**

To ensure a smooth and consistent start, especially at $t=0$, we will initialize the temperature field across the entire domain to a value that satisfies the nonlinear right boundary condition. Specifically:
$$u(x, 0) = \sqrt{u_{\text{target\_squared}}} = 10$$
This ensures that $u(0,0)^2 = u_{\text{target\_squared}}$ is initially true, providing a well-defined starting point for the nonlinear solver.


### 3.2 Solution Approaches

The problem is time-dependent. We will discretize the time derivative using a Backward Euler scheme, which is implicitly stable. At each time step $t_n$, this leads to a nonlinear algebraic system for the unknown solution $u_n$.

The weak form of the equation at time $t_n$ (using a Backward Euler approximation for $\frac{\partial u}{\partial t} \approx \frac{u_n - u_{n-1}}{\Delta t}$) is:

$$\int_{\Omega} \frac{u_n - u_{n-1}}{\Delta t} v \, dx + \int_{\Omega} k \frac{du_n}{dx} \frac{dv}{dx} \, dx = 0$$

Rearranging this into a residual form $R(u_n) = 0$:

$$R(u_n; u_{n-1}, \Delta t) = \int_{\Omega} \left( \frac{u_n}{\Delta t} v + k \frac{du_n}{dx} \frac{dv}{dx} \right) \, dx - \int_{\Omega} \frac{u_{n-1}}{\Delta t} v \, dx$$

To solve this, we will leverage the capabilities of `DifferentialEquations.jl`:

1.  Formulate the Nonlinear System via `ODEFunction`: We define an `ODEFunction` that represents the residual $R(u_n)$ and its Jacobian $J(u_n) = \frac{\partial R}{\partial u_n}$.
    * The time derivative term $\frac{u_n}{\Delta t} v$ (from $M u_n / \Delta t$) is handled by providing a `mass_matrix` to the `ODEFunction`.
    * The diffusion term and the direct enforcement of the nonlinear boundary condition at $x=0$ will be implemented within our `ode_function!` (for $R$) and `ode_jacobian!` (for $J$).
        * For a DOF $i$ at $x=0$:
            * $R_i = (u_i)^2 - 100$
            * $J_{ii} = 2 u_i$
            * $J_{ij} = 0 \quad \text{for } j \neq i$ (on that row).

2.  Implicit Solver with Built-in Newton-Raphson: We will then pass this `ODEFunction` to an implicit ordinary differential equation (ODE) solver from `DifferentialEquations.jl`, such as `Rodas5P`.

This strategy combines the power of Finite Element Method (FEM) for spatial discretization, Backward Euler for temporal discretization, and the advanced nonlinear solving capabilities of `DifferentialEquations.jl`, providing a stable and accurate solution from $t=0$.

### 3.3 Numerical Solution

In [26]:
# --- 1. Mesh Generation and Finite Element Setup ---
const dim = 1 # Problem dimension: 1D
const L = 1.0 # Length of the domain
const nel = 20 # Number of elements
grid = generate_grid(Line, (nel,)) # Generate a 1D grid of 'nel' Line elements

# Interpolation (P2 for the scalar field 'u')
ip_u = Lagrange{RefLine, 2}() # Lagrange interpolation of degree 2 for a Line element

# Quadrature rules for integration
qr_cell = QuadratureRule{RefLine}(3) # 3 Gauss points for cell (volume) integrals
qr_facet = FacetQuadratureRule{RefLine}(2) # 2 Gauss points for facet (point) integrals

# CellValues and FacetValues for the scalar field 'u'
cv_u = CellValues(qr_cell, ip_u); 
fv_u = FacetValues(qr_facet, ip_u); 

In [27]:
# --- 2. Problem Parameters ---
const k_conductivity = 1.0 # Thermal conductivity (or diffusion coefficient)
const u_left_target_squared = 100.0 # Target for the nonlinear left BC: u(0,t)^2 = 100

# Time stepping parameters
const T_END = 10.0 # Total simulation time
const Δt = 0.1 # Fixed time step size
const num_timesteps = ceil(Int, T_END / Δt)

# Initial condition for the field 'u' (consistent with the nonlinear left BC at t=0)
const initial_u_value = u_left_target_value; # Initialize u everywhere to sqrt(u_left_target_squared) (e.g., 10.0)

In [28]:
# --- 3. DofHandler and ConstraintHandler Setup ---
# DofHandler: manages the distribution of DOFs for the scalar field 'u'
dh = DofHandler(grid)
add!(dh, :u, ip_u) 
close!(dh);

# ConstraintHandler: manages strong boundary conditions (for the right boundary)
ch = ConstraintHandler(dh);

# Define node sets for boundaries
left_node_set  = getfacetset(dh.grid, "left")  # Node at x=0.0
right_node_set = getfacetset(dh.grid, "right") # Node at x=L

# Create Dirichlet BCs for u=0 on these patches
dbc_right = Dirichlet(:u, right_node_set, (x,t) -> 0.0)


# Add these new Dirichlet BCs to the ConstraintHandler
add!(ch, dbc_right)

close!(ch); 

In [30]:
# --- Utility functions for debugging ---
function check_for_nans_and_infs(matrix, vector, context_message::String)
    matrix_vals = typeof(matrix) <: SparseMatrixCSC ? nonzeros(matrix) : matrix
    
    if any(isnan, matrix_vals) || any(isinf, matrix_vals)
        @error "NaNs or Infs detected in Jacobian ($context_message)!"
        error("Jacobian contains NaNs/Infs.")
    end
    if any(isnan, vector) || any(isinf, vector)
        @error "NaNs or Infs detected in Residual ($context_message)!"
        error("Residual contains NaNs/Infs.")
    end
end

# --- 4. Function for Initial Conditions ---
# This function initializes U0 with u=initial_u_value everywhere
function setup_initial_conditions!(dh::DofHandler, initial_u_val::Float64)
    U0 = zeros(ndofs(dh)) 

    # Initialize the field :u everywhere with initial_u_val
    apply_analytical!(U0, dh, :u, x -> initial_u_val)
    
    # DEBUG PRINTS for U0 (will only print once at start of simulation)
    println("\n--- DEBUG: U0 after setup_initial_conditions! (via apply_analytical!) ---")
    println("U0 (truncated to first 20 elements): ", U0[1:min(20, end)]) 
    println("Size of U0: ", length(U0))
    println("-----------------------------------------------\n")
    
    return U0
end


# --- 5. Assembly Function for the Nonlinear Algebraic System at each Timestep ---
# This function assembles the global Residual (R_global) and Jacobian (J_global)
# for the nonlinear system: R_global(u_n) = 0, where u_n is the solution at current time.
# R_global(u_n) = (M/Δt + K)u_n - M u_{n-1}/Δt
# It directly incorporates the nonlinear left BC and applies the linear right BC.
function assemble_nonlinear_system!(J_global::SparseMatrixCSC, R_global::Vector,
                                      cv_u::CellValues, fv_u::FacetValues,
                                      dh::DofHandler, ch::ConstraintHandler, # Pass ch to apply strong BCs
                                      u_current_iterate::Vector, # Current Newton iterate for u_n
                                      u_prev_timestep::Vector,   # Solution u_{n-1} from previous time step
                                      Δt::Float64,
                                      left_node_set,
                                      u_left_target_squared::Float64)
                                      
    fill!(nonzeros(J_global), 0.0) # Reset J_global and R_global for current assembly
    fill!(R_global, 0.0)
    
    assembler = start_assemble(J_global, R_global)

    n_basefuncs = getnbasefunctions(cv_u) 
    n_el_dofs = n_basefuncs # Number of DOFs per element for scalar field
    
    Je = zeros(n_el_dofs, n_el_dofs) # Elementary Jacobian matrix
    re = zeros(n_el_dofs)            # Elementary residual vector

    for cell_idx in 1:getncells(dh.grid)
        cell = dh.grid.cells[cell_idx]
        fill!(Je, 0.0) 
        fill!(re, 0.0) 

        Ferrite.reinit!(cv_u, cell) 

        global_dofs_cell = celldofs(cell) 
        
        # Extract local DOF values for the current Newton iterate (u_n) and previous time step (u_{n-1})
        uloc_current = u_current_iterate[global_dofs_cell]
        uloc_prev = u_prev_timestep[global_dofs_cell]

        for q_point in 1:getnquadpoints(cv_u) 
            dΩ = getdetJdV(cv_u, q_point) 

            # Evaluate current solution and its gradient at QP
            u_qp_current = function_value(cv_u, q_point, uloc_current) 
            ∇u_qp_current = function_gradient(cv_u, q_point, uloc_current) 

            # Evaluate previous solution at QP
            u_qp_prev = function_value(cv_u, q_point, uloc_prev)

            # --- Contributions from the Heat Equation (Domain Terms) ---
            # Residual term: (u_n / Δt * v_i) + (k * ∇u_n ⋅ ∇v_i) - (u_{n-1} / Δt * v_i)
            # This is effectively (M_e/Δt + K_e) * uloc_current - M_e/Δt * uloc_prev for residual
            # And (M_e/Δt + K_e) for Jacobian
            for i in 1:n_basefuncs # Loop over test functions v_i
                v_i = shape_value(cv_u, q_point, i)    
                ∇v_i = shape_gradient(cv_u, q_point, i)    
                
                re[i] += ((u_qp_current / Δt) * v_i + k_conductivity * (∇u_qp_current ⋅ ∇v_i) - (u_qp_prev / Δt) * v_i) * dΩ
                
                for j in 1:n_basefuncs # Loop over trial functions phi_j
                    φ_j = shape_value(cv_u, q_point, j)
                    ∇φ_j = shape_gradient(cv_u, q_point, j)
                    
                    # Jacobian term: ∂R_i / ∂u_j = (1/Δt * v_i * φ_j) + (k * ∇v_i ⋅ ∇φ_j)
                    Je[i, j] += ((1.0 / Δt) * v_i * φ_j + k_conductivity * (∇v_i ⋅ ∇φ_j)) * dΩ
                end
            end
        end

        # Assemble elementary contributions into global matrices
        assemble!(assembler, global_dofs_cell, Je, re) 
    end
    
    # --- Direct Enforcement of Nonlinear Left Boundary Condition ---
    # Find the global DOF for the left boundary node (only one for a 1D grid at x=0)
    # The `only` function is used because `left_node_set` should contain exactly one NodeIndex.
    left_boundary_dof_idx = Ferrite.dof_cells(dh, only(left_node_set))[1].dof 
    
    # Get the current iterate's value at the left boundary node
    u_left_boundary_iterate = u_current_iterate[left_boundary_dof_idx]

    # Override the residual for this DOF: R_i = u_i^2 - u_target_squared
    R_global[left_boundary_dof_idx] = u_left_boundary_iterate^2 - u_left_target_squared

    # Override the Jacobian row for this DOF: J_ii = 2*u_i, J_ij = 0 (j != i)
    # Clear the entire row in the sparse matrix corresponding to this DOF
    # Important: Ferrite's `create_sparsity_pattern` should ensure the diagonal element exists.
    for col_idx in J_global.colptr[left_boundary_dof_idx]:(J_global.colptr[left_boundary_dof_idx+1]-1)
        if J_global.rowval[col_idx] == left_boundary_dof_idx # Check if it's the diagonal entry
            J_global.nzval[col_idx] = 2.0 * u_left_boundary_iterate
        else
            J_global.nzval[col_idx] = 0.0 # Set off-diagonal entries in this row to 0
        end
    end

    # --- Apply Strong Dirichlet BCs for the Right Boundary (u(L,t) = 0.0) ---
    # This sets R_global[dof] = 0 and J_global[dof, dof] = 1 for the constrained DOFs,
    # and zeroes out other entries in the constrained rows/columns.
    Ferrite.apply!(J_global, R_global, ch)

    return J_global, R_global
end


# --- 6. Newton-Raphson Solver for Nonlinear Algebraic System at each Timestep ---
# This function encapsulates the Newton-Raphson iterations to find u_n for a given time step.
function solve_nonlinear_system_at_timestep!(u_n::Vector, u_prev::Vector, Δt::Float64,
                                             J_global_preallocated::SparseMatrixCSC,
                                             R_global_preallocated::Vector,
                                             cv_u::CellValues, fv_u::FacetValues, 
                                             dh::DofHandler, ch::ConstraintHandler,
                                             left_node_set, 
                                             u_left_target_squared::Float64)
    
    # Initial guess for Newton-Raphson is the solution from the previous time step
    u_current_iterate = copy(u_prev) 

    max_newton_iterations = 100
    newton_tol = 1e-8 # Tolerance for the residual norm
    
    println("\n--- Starting Newton-Raphson iteration for current time step ---")

    for n_iter in 1:max_newton_iterations
        # Assemble Residual and Jacobian for the current iterate
        J_current, R_current = assemble_nonlinear_system!(J_global_preallocated, R_global_preallocated,
                                                            cv_u, fv_u, dh, ch, 
                                                            u_current_iterate, u_prev, Δt,
                                                            left_node_set, u_left_target_squared)
        
        check_for_nans_and_infs(J_current, R_current, "Newton iteration $(n_iter)")

        norm_R = norm(R_current)
        @printf "Newton iteration %3d | Residual norm: %e\n" n_iter norm_R

        # Check for convergence
        if norm_R < newton_tol
            println("Newton-Raphson converged in $n_iter iterations.")
            u_n .= u_current_iterate # Update u_n with the converged solution
            return true # Indicate success
        end

        # Solve for correction: J * Δu = -R
        Δu = J_current \ (-R_current) # This performs the linear solve
        
        # Update current iterate
        u_current_iterate .+= Δu

        # After each Newton step, re-apply the STRONG BCs (like u(L,t)=0)
        # This is crucial to ensure the BCs are always satisfied by the iterate.
        Ferrite.apply!(u_current_iterate, ch) 
    end

    @warn "Newton-Raphson did NOT converge after $max_newton_iterations iterations."
    u_n .= u_current_iterate # Return the last iterate even if not converged
    return false # Indicate failure
end

# --- 7. Main Time Stepping Loop ---
# This function drives the entire simulation by iterating through time steps.
function solve_heat_diffusion_1d_nonlinear_bc(dh::DofHandler, ch::ConstraintHandler, 
                                               cv_u::CellValues, fv_u::FacetValues, 
                                               left_node_set, 
                                               right_node_set,
                                               u_left_target_squared::Float64)
    
    ndofs_total = ndofs(dh) 
    
    # Initialize u_prev (u_0) for the very first time step
    u_prev = setup_initial_conditions!(dh, initial_u_value) # u(x,0) = 10 everywhere

    # Pre-allocate global Jacobian and Residual for Newton-Raphson
    J_global_preallocated = create_sparsity_pattern(dh) 
    R_global_preallocated = zeros(ndofs_total)

    # The initial condition u_prev needs to satisfy ALL strong BCs, even at t=0.
    # The right boundary (u(L,t)=0) is a strong BC, so apply it here for initial state.
    Ferrite.apply!(u_prev, ch) # This will set u_prev[right_boundary_dofs] = 0.0 at t=0

    u_n = similar(u_prev) # Will hold the solution for the current time step u_n

    println("\n--- Starting Time Stepping Simulation ---")
    current_time = 0.0

    # Store solutions at each time step (optional, for plotting time history)
    solutions_at_timesteps = [copy(u_prev)] 
    times_recorded = [0.0]

    for i_ts in 1:num_timesteps
        current_time += Δt
        @printf "Time Step %3d | Current Time: %10.4f\n" i_ts current_time

        # Solve the nonlinear system at current_time using u_prev as initial guess
        converged = solve_nonlinear_system_at_timestep!(u_n, u_prev, Δt,
                                                       J_global_preallocated, R_global_preallocated,
                                                       cv_u, fv_u, dh, ch, 
                                                       left_node_set, u_left_target_squared)
        
        if !converged
            @warn "Time step $i_ts did not converge. Aborting simulation."
            break
        end

        u_prev .= u_n # Update u_prev for the next time step
        push!(solutions_at_timesteps, copy(u_n)) # Store current solution
        push!(times_recorded, current_time)

    end

    println("\n--- Simulation Finished ---")
    return u_n, solutions_at_timesteps, times_recorded # Return the final converged solution and history
end

# --- 8. Main Execution ---
# Call the main solver function
final_solution_u, solutions_history, times_history = solve_heat_diffusion_1d_nonlinear_bc(dh, ch, cv_u, fv_u, left_node_set, right_node_set, u_left_target_squared);

println("Final 1D Heat Diffusion solution obtained.")
println("Total DOFs: ", ndofs(dh))

u_final_nodal = Ferrite.get_field(final_solution_u, dh, :u)

println("Min u: ", minimum(u_final_nodal))
println("Max u: ", maximum(u_final_nodal))

# For a 1D grid `generate_grid(Line, (nel,))`, nodes are ordered from left to right.
# So, the first DOF (index 1) is at x=0, and the last DOF is at x=L.
# This assumes that the `DofHandler` assigns DOFs monotonically with node index.
println("u(x=0) final (expected sqrt(100) or 10): ", u_final_nodal[1]) 
println("u(x=L) final (expected 0.0): ", u_final_nodal[end]) 



--- DEBUG: U0 after setup_initial_conditions! (via apply_analytical!) ---
U0 (truncated to first 20 elements): [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
Size of U0: 41
-----------------------------------------------



LoadError: DeprecationError: [31m`create_sparsity_pattern(args...)`[39m is deprecated, use [32m`allocate_matrix(args...; kwargs...)`[39m instead.